In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [ ]:
df_raw=pd.read_csv('../raw_data/titanic.csv',index_col=0)
df = df_raw.copy()

df['Age'].fillna(df['Age'].mean(),inplace=True)

df_main=df.loc[:,['Fare','Sex','Pclass','Age','Survived']]

data = df_main.iloc[:,:-1]
target = df_main.iloc[:,-1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe_sex = LabelEncoder()
data['Sex']=lbe_sex.fit_transform(data['Sex'])

In [ ]:
from sklearn.preprocessing import RobustScaler
rbs = RobustScaler()
data_rbs=rbs.fit_transform(data)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(data_rbs,target,test_size=0.2,stratify=target)

In [ ]:
import numpy as np
test_arr=np.array([30,0,1,25])
test_arr_rbs=rbs.transform(test_arr.reshape(1,-1))

### Modeling
```
 1. tf_low
 2. tf_high
 3. torch_low
 4. torch_high
 ```

In [ ]:
# 1
import tensorflow as tf

x = tf.constant(x_train, tf.float32)
# y = tf.constant(target.values.reshape(-1,1), tf.float32)
y = tf.constant(y_train.values.reshape(-1,1), tf.float32)

initX = tf.initializers.GlorotUniform()
w = tf.Variable(initX(shape=[x.shape[-1],1]))
b = tf.Variable(initX(shape=[1]))

def cost():
    z = tf.matmul(x,w) + b
    c_i = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=z)
    c = tf.reduce_mean(c_i)
    return c

def hxFn(xdata):
    xd = tf.constant(xdata,tf.float32)
    z = tf.matmul(xd,w) + b
    hx = tf.nn.sigmoid(z)
    return (hx.numpy()>0.5) + 0

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

for i in range(1000):
    optimizer.minimize(cost,var_list=[w,b])
    acc = accuracy_score(hxFn(x),y)
    print(f'epoch => {i} / cost => {cost().numpy()} / acc => {acc}')

In [ ]:
print("f1_score : ",f1_score(y_test,hxFn(x_test)))
sns.heatmap(
    confusion_matrix(y_test,hxFn(x_test)),
    annot=True,
    annot_kws={"size": 20},
    fmt=".0f")
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()
pred=hxFn(tf.constant(test_arr_rbs, tf.float32))
print("30달러 여성, 1등급 25세 : ", pred ,' => 생존')

In [ ]:
# 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=1,
                input_dim=data.shape[-1],
                activation='sigmoid'
                ))
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1),
    metrics=['acc']
)
hist = model.fit(data_rbs,target,epochs=50)

In [ ]:
pred=model.predict(test_arr_rbs)

In [ ]:
(pred>0.5)+0

In [ ]:
print("f1_score : ",f1_score(y_test,(model.predict(x_test)>0.5)+0))
sns.heatmap(
    confusion_matrix(y_test,(model.predict(x_test)>0.5)+0),
    annot=True,
    annot_kws={"size": 20},
    fmt=".0f")
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()
pred=(model.predict(test_arr_rbs)>0.5)+0
print("30달러 여성, 1등급 25세 : ", pred ,' => 생존')

In [ ]:
# 3
import torch
from torch.nn import functional as F

x = torch.FloatTensor(data_rbs)
y = torch.FloatTensor(target.values.reshape(-1,1))

w = torch.empty([x.shape[-1],1],requires_grad=True)
b = torch.empty([1],requires_grad=True)
torch.nn.init.uniform_(w)
torch.nn.init.uniform_(b)

def cost():
    z = torch.matmul(x,w) + b
    c_i = F.binary_cross_entropy_with_logits(z,y)
    c = torch.mean(c_i)
    return c

def hxFn(xdata):
    xd = torch.FloatTensor(xdata)
    z = torch.matmul(xd,w) + b
    hx = torch.sigmoid(z)
    return (hx.detach().numpy() > 0.5) + 0

optimizer = torch.optim.Adam([w,b],lr=0.1)

for i in range(1000):
    c=cost()
    optimizer.zero_grad()
    c.backward()
    optimizer.step()
    acc = accuracy_score(target,hxFn(data_rbs))
    print(f'epoch => {i} / cost => {cost().item()} acc => {acc} ')

In [ ]:
print("f1_score : ",f1_score(y_test,hxFn(x_test)))
sns.heatmap(
    confusion_matrix(y_test,hxFn(x_test)),
    annot=True,
    annot_kws={"size": 20},
    fmt=".0f")
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()
pred=hxFn(test_arr_rbs)
print("30달러 여성, 1등급 25세 : ", pred ,' => 생존')

pred=hxFn(test_arr_rbs)
print(pred)

In [ ]:
#4
from torch.optim import Adam
from torch.nn import BCELoss # Binary Cross Entropy


x = torch.FloatTensor(data_rbs)
y = torch.FloatTensor(target.values.reshape(-1,1))

model =torch.nn.Sequential()
model.add_module('nn1',torch.nn.Linear(x.shape[-1],1))
model.add_module('sig1',torch.nn.Sigmoid())
loss_fn = BCELoss()
optimizer = Adam( model.parameters(), lr=0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    hx = model(x)
    """
    => model.forward(x)
    => z = matmul(x,w) + b : Linear
    => Sigmoid(z) : Sigmoid
    """
    c = loss_fn(hx,y)
    c.backward()
    optimizer.step()
    pred=(hx.detach().numpy() > 0.5) +0
    print(f'epoch => {epoch} / cost => {c.detach().numpy()} \
        acc => {accuracy_score(pred,y)} ')

In [ ]:
print("f1_score : ",f1_score(y_test,hxFn(x_test)))
sns.heatmap(
    confusion_matrix(y_test,hxFn(x_test)),
    annot=True,
    annot_kws={"size": 20},
    fmt=".0f")
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()
pred=hxFn(test_arr_rbs)
print("30달러 여성, 1등급 25세 : ", pred ,' => 생존')